In [197]:
from bert_serving.client import BertClient
import pandas as pd
import numpy as np
from sklearn import datasets
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.decomposition import PCA
from sklearn.metrics import accuracy_score


In [2]:
bc = BertClient()

In [126]:
filename = '新华社数据.csv'
articles = pd.read_csv(filename, encoding='gb18030')
contents = articles['content'].tolist()[:10000]
sources = articles['source'].tolist()[:10000]

In [128]:
Y = sources
for i in range(len(sources)):
    if sources[i] == '新华社':
        Y[i] = 1
    else:
        Y[i] = 0

In [106]:
X = []
for i in range(len(contents)):
    if i%100 == 0:
        print(i)
    x = bc.encode([contents[i]]).tolist()[0]
    X.append(x)

0
100
200
300
400
500
600
700
800
900
1000


C:\ProgramData\Anaconda3\lib\site-packages\bert_serving\client\__init__.py:299: UserWarning: some of your sentences have more tokens than "max_seq_len=300" set on the server, as consequence you may get less-accurate or truncated embeddings.
here is what you can do:
- disable the length-check by create a new "BertClient(check_length=False)" when you do not want to display this warning
- or, start a new server with a larger "max_seq_len"
  '- or, start a new server with a larger "max_seq_len"' % self.length_limit)


1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900


## SVM

In [142]:
clf= SVC()

In [152]:
clf.fit(X,Y)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\svm\base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
  kernel='rbf', max_iter=-1, probability=False, random_state=None,
  shrinking=True, tol=0.001, verbose=False)

In [184]:
def predict(news):
    vec = bc.encode([news]).tolist()[0]
    result = clf.predict([vec])[0]
    if result == 1:
        return '新华社新闻'
    else:
        return '非新华社新闻'

In [185]:
news = articles['content'][30000]
predict(news)

'新华社新闻'

In [186]:
articles['source'][30000]

'新华社'

In [187]:
clf.score(X,Y)

0.9614

In [193]:
contents_test = articles['content'].tolist()[10001:10101]
X_test = []
for i in range(len(contents_test)):
    if i%10 == 0:
        print(i)
    x_test = bc.encode([contents_test[i]]).tolist()[0]
    X_test.append(x_test)
Y_test_ = articles['source'].tolist()[10001:10101]
Y_test = Y_test_
for i in range(len(Y_test_)):
    if Y_test_[i] == '新华社':
        Y_test[i] = 1
    else:
        Y_test[i] = 0
clf.score(X_test,Y_test)

0
10
20
30
40
50
60
70
80
90


0.92

## RandomForest

In [188]:
rnd_clf = RandomForestClassifier(n_estimators=500)

In [189]:
rnd_clf.fit(X,Y)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=500, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [194]:
def predict_rf(news):
    vec = bc.encode([news]).tolist()[0]
    result = rnd_clf.predict([vec])[0]
    if result == 1:
        return '新华社新闻'
    else:
        return '非新华社新闻'

In [195]:
predict_rf(news)

'新华社新闻'

In [198]:
rnd_clf.score(X,Y)

0.9998

In [199]:
rnd_clf.score(X_test,Y_test)

0.79

## XGBoost

In [202]:
xg = XGBClassifier()

In [203]:
X_XGB = np.array(X)
Y_XGB = np.array(Y)

In [205]:
xg.fit(X_XGB,Y_XGB)

XGBClassifier(base_score=0.5, booster=None, colsample_bylevel=1,
       colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
       importance_type='gain', interaction_constraints=None,
       learning_rate=0.300000012, max_delta_step=0, max_depth=6,
       min_child_weight=1, missing=nan, monotone_constraints=None,
       n_estimators=100, n_jobs=0, num_parallel_tree=1,
       objective='binary:logistic', random_state=0, reg_alpha=0,
       reg_lambda=1, scale_pos_weight=1, subsample=1, tree_method=None,
       validate_parameters=False, verbosity=None)

In [240]:
def predict_XGB(news):
    vec_ = bc.encode([news]).tolist()[0]
    vec = np.array([vec_])
    print(type(vec))
    result = xg.predict(vec)
    if result[0] == 1:
        return '新华社新闻'
    else:
        return '非新华社新闻'

In [241]:
predict_XGB(news)

<class 'numpy.ndarray'>


'新华社新闻'

In [247]:
xg.score(X_XGB,Y_XGB)

0.9998

In [251]:
xg.score(np.array(X_test),np.array(Y_test))

0.91

| -|SVM|rnd|XGB|
|---|---|---|---|
|score_train|0.9614|0.9998|0.9998|
|score_test|0.92|0.79|0.91|